In [29]:
#!/usr/bin/env python
import sys
import os
import pickle as pkl
import warnings
import time

from functools import wraps
from pathlib import Path
from multiprocessing import cpu_count
from multiprocessing.pool import Pool as Pool

import numpy as np
import matplotlib.pyplot as plt

#sys.path.insert(0,'/groups/astro/robinboe/mass_analysis')
#sys.path.insert(0,'/groups/astro/robinboe/mass_analysis/computable-information-density')
#from joblib import Parallel, delayed

from ComputableInformationDensity.cid import interlaced_time, cid2d
from ComputableInformationDensity.computable_information_density import cid

# Get absolute path to this notebook
notebook_path = Path().resolve()
parent_dir = notebook_path.parent
sys.path.append(str(parent_dir))
from NematicAnalysis.utils import gen_clustering_metadata, get_defect_arr_from_frame

In [136]:
def timeit(func):
    @wraps(func)   # keeps function name/docs intact
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)   # call the real function
        end = time.perf_counter()
        print(f"{func.__name__} runtime: {end - start:.3f} s")
        return result
    return wrapper

def get_allowed_time_intervals(system_size, nbits_max = 8):
    """
    Get allowed intervals for CID calculation based on system size and max bits.
    """
    allowed_intervals = []
    
    # system size must be divisible by 2^n

    if np.log2(system_size) % 1 != 0:
        warnings.warn("System size must be a power of 2 for exact interval calculation.")
        return
    if not type(nbits_max) == int:
        raise ValueError("nbits_max must be an integer.")

    for nbits in range(1, nbits_max + 1):
        interval_exp = 3 * nbits - 2 * np.log2(system_size)

        if interval_exp < 0:
            continue

        allowed_intervals.append({'time_interval': int(2 ** interval_exp), 'nbits': nbits})
    return allowed_intervals

def lz77py(sequence):
    """ Lempel-Ziv 77 complexity \n
    Note: The input sequence is cast as string 
    and not tuple (for performance reasons).
    """
    seq = ''.join(map(str, sequence))
    complexity, ind, inc = 1, 1, 0
    while ind + inc < len(seq):
        if seq[ind : ind + inc + 1] in seq[: ind + inc]:
            inc += 1
        else:
            complexity += 1
            ind += inc + 1
            inc = 0
    return complexity + 1 if inc != 0 else complexity

#@timeit
def block_flatten(array, m, k):
    """
    Efficiently flatten a 2D array into m x k blocks traversed horizontally.
    
    Parameters:
        array (np.ndarray): Input 2D array of shape (M, N)
        m (int): Number of rows per block
        k (int): Number of columns per block
        
    Returns:
        np.ndarray: Flattened 1D array of blocks
    """
    M, N = array.shape

    # Check divisibility
    if M % m != 0:
        raise ValueError(f"Number of rows {M} is not divisible by block row size {m}.")
    if N % k != 0:
        raise ValueError(f"Number of columns {N} is not divisible by block column size {k}.")
    
    # Reshape array into blocks
    reshaped = array.reshape(M//m, m, N//k, k)
    # Transpose to bring blocks into row-major order: (block_row, block_col, m, k)
    reshaped = reshaped.transpose(0, 2, 1, 3)
    # Flatten all blocks
    return reshaped.reshape(-1)


def _shuffle_generator(seq, nshuff, rng):
    """Yield reshuffled copies of a working array."""
    arr = seq.copy()  # one upfront copy
    for _ in range(nshuff):
        rng.shuffle(arr)        # shuffle in place
        yield arr.copy()        # copy here so workers don't share buffer

#@timeit
def cid_shuffle_dev(sequence, nshuff, ddof = 1):
    """Computable Information Density via random shuffling.
    
    Args:
        sequence : np.ndarray
            One-dimensional data array.
        nshuff   : int
            Number of shuffles.
    Returns:
        float : Mean CID value over shuffled sequences.
    """
    rng = np.random.default_rng()  # independent RNG

    # Generate reshuffled arrays lazily
    shuffled_iter = _shuffle_generator(sequence, nshuff, rng)

    with Pool(min(cpu_count(), nshuff)) as pool:
        cid_values = pool.map(cid, shuffled_iter)  # cid must be top-level
    return np.mean(cid_values), np.std(cid_values, ddof=ddof if nshuff > 1 else 0)

In [3]:
base_path = 'C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\na512'
d_cluster_l = dict(path = base_path + 'l_cl', \
              suffix = "l_cl", priority = 1, LX = 512, Nframes = 400)
d_cluster_vl = dict(path = base_path + 'vl_cl', \
              suffix = "vl_cl", priority = 1, LX = 512, Nframes = 1500)
d_list = [d_cluster_l, d_cluster_vl]

Nexp_l, act_l, act_dir_l = gen_clustering_metadata(d_cluster_l['path'])
Nframes_l = 400
Nexp_vl, act_vl, act_dir_vl = gen_clustering_metadata(d_cluster_vl['path'])
Nframes_vl = 1500

## Choose which dataset to visualize
act = 0.022
act_idx = act_vl.index(act)
num_exp = 0
rmax = 33
folder_path = os.path.join(act_dir_vl[act_idx], f'zeta_{act}_counter_{num_exp}')
defect_path = os.path.join(folder_path, 'defect_positions.pkl')
labels_path = os.path.join(folder_path, 'labels_rm33.pkl')

with open(defect_path, 'rb') as f:
    defect_list = pkl.load(f)
with open(labels_path, 'rb') as f:
    labels = pkl.load(f)

In [ ]:
nframes, nx, ny = 8, d_cluster_l['LX'], d_cluster_l['LX']

allowed_intervals_list = get_allowed_time_intervals(system_size = nx, nbits_max=8)

# check that nframes is in allowed intervals
if nframes not in [ai['time_interval'] for ai in allowed_intervals_list]:
    raise ValueError(f"nframes {nframes} is not in allowed intervals {allowed_intervals_list}")
else:
    # get nbits for nframes
    nbits = [ai['nbits'] for ai in allowed_intervals_list if ai['time_interval'] == nframes][0]

defect_grid = np.zeros((nframes, nx, ny), dtype=int)
defect_density = []

for i, defect in enumerate(defect_list[:nframes]):
    def_arr = get_defect_arr_from_frame(defect).astype(int)

    defect_grid[i, def_arr[:,0], def_arr[:,1]] = 1
    defect_density.append(defect_grid[i,:,:].mean())



In [127]:
get_allowed_time_intervals(system_size = 32, nbits_max=8)

[{'time_interval': 4, 'nbits': 4},
 {'time_interval': 32, 'nbits': 5},
 {'time_interval': 256, 'nbits': 6},
 {'time_interval': 2048, 'nbits': 7},
 {'time_interval': 16384, 'nbits': 8}]

In [137]:
   
N = 16
M = 16
kN = 1

system_size = N + M
nframes = 32

allowed_intervals_list = get_allowed_time_intervals(system_size = system_size, nbits_max=8)

# check that nframes is in allowed intervals
if nframes not in [ai['time_interval'] for ai in allowed_intervals_list]:
    raise ValueError(f"nframes {nframes} is not in allowed intervals {allowed_intervals_list}")
else:
    # get nbits for nframes
    nbits = [ai['nbits'] for ai in allowed_intervals_list if ai['time_interval'] == nframes][0]

np0, np1 = -1,9
mblock, nblock = 2,2

npoints = 1000
nshuffle=32

# construct array with order along both axes
X=np.block([[
    np.zeros((N,kN*N)),
    np.ones((N,kN*N))],
    [0*np.ones((M,kN*N)),
     0*np.ones((M,kN*N)),]
    ]).astype(int)
# contruct array with order along one axis
Xpermute = X.astype(int) #np.roll(X, shift=-2, axis=0)
Xpermute[[0,np0],:] = X[[np0,0],:]
Xpermute[[1, np1],:] = X[[np1,1],:]

Xt = np.repeat(X[None,:,:],nframes,axis=0)   #np.dstack([X]*nframes,axis=0)
Xt = np.repeat(Xpermute[None,:,:],nframes,axis=0)   #np.dstack([X]*nframes,axis=0)
print(Xt.shape, X.shape)
print(X)

(32, 32, 32) (32, 32)
[[0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [138]:
# instantiate CID object:
nshuffle=4
mblock, kblock = 2,2
t1 = time.perf_counter()
CID = interlaced_time(nbits=nbits, nshuff=nshuffle)

cid_, cid_shuff = CID(Xt.copy())
print(f"Interlaced: CID, CID_shuff, ratio: {cid_}, {cid_shuff}", cid_/cid_shuff)
t2 = time.perf_counter()
print(f"Time taken: {(t2 - t1):.1f} seconds")
CID2d = cid2d(nbits=nbits, nshuff=nshuffle)
cid2d_, cid2d_shuff = CID2d(Xt.copy())
print(f"cid2d: CID, CID_shuff, ratio: {cid2d_},{cid2d_shuff}", cid2d_/cid2d_shuff)
t2 = time.perf_counter()
print(f"Time taken: {(t2 - t1):.1f} seconds")

cid_list = []
cid_list_block = []
cid_shuffle_list_block = []
cid_shuffle_list = []
for i in range(nframes):
    Xblock = block_flatten(Xt[i,:,:], m=mblock, k=nblock)
    cid_list_block.append(cid(Xblock.astype(int).flatten()))
    cid_shuffle_list_block.append(cid_shuffle_dev(Xblock.flatten(), nshuff=nshuffle)[0])
    cid_list.append(cid(Xt[i,:,:].astype(int).flatten()))
    cid_shuffle_list.append(cid_shuffle_dev(Xt[i,:,:].flatten(), nshuff=nshuffle)[0])
print("Sequential: CID, CID_shuff, ratio: ", np.mean(cid_list), np.mean(cid_shuffle_list), np.mean(cid_list)/ np.mean(cid_shuffle_list),)
print("Block flattened: CID, CID_shuff, ratio: ", np.mean(cid_list_block), np.mean(cid_shuffle_list_block), np.mean(cid_list_block)/ np.mean(cid_shuffle_list_block),)
print("time taken: ", (time.perf_counter() - t2)/2)

Interlaced: CID, CID_shuff, ratio: 0.0751149831507952, 1.046482492358989 0.07177853781525807
Time taken: 5.9 seconds
cid2d: CID, CID_shuff, ratio: 0.013627858941792376,1.0440436622134668 0.013052958832104862
Time taken: 14.8 seconds
Sequential: CID, CID_shuff, ratio:  0.10204123534733699 1.1535442938006686 0.08845887920882006
Block flattened: CID, CID_shuff, ratio:  0.14792058104201092 1.1542813433934853 0.12814950348858403
time taken:  13.283701700000165


In [ ]:
dim=3
bits_max = 8
lx = 16
allowed_int_idx = 1

allowed_intervals = get_allowed_time_intervals(lx, bits_max)

nframes=allowed_intervals[allowed_int_idx]['time_interval']
nbits = allowed_intervals[allowed_int_idx]['nbits']
size = 1 << allowed_intervals[allowed_int_idx]['nbits']
print(f"nframes: {nframes}, nbits: {nbits}, size: {size}"   )
X=np.ones((nframes, lx, lx))



nframes: 16, nbits: 4, size: 16
CID interlaced: 0.021484375, CID shuffled interlaced: 0.021484375
Time taken: 1.5 seconds
CID 2d: 0.8321814137333032, CID shuffled 2d: 0.8237646426378623
Time taken: 2.8 seconds


In [114]:
data = np.transpose(Xt).reshape((-1, ) + (1<<nbits, ) * 3).T
X.shape, data.shape

((16, 16), (8, 8, 8, 1))

In [ ]:
nframes, nx, ny = 512, 1024, 1024

# instantiate CID object:
CID = interlaced_time(8, 16)

defects = zeros((nframes, nx, ny), dtype=int)
defect_density = []

ar = archive.loadarchive(f'/lustre/astro/kpr279/ns2048/output_test_zeta_{zeta}/output_test_zeta_{zeta}_counter_{rep}')

print(zeta,rep)

for i in range(nframes):
    frame = ar[i]
    for defect in get_defects(frame.QQxx, frame.QQyx, frame.LX, frame.LY):
        ix, iy = defect['pos']
        ix, iy = int(ix) - nx//2, int(iy) - ny//2
        if (0 <= ix < nx) and (0 <= iy < ny): defects[i, ix, iy] = 1

    defect_density.append(defects[i,:,:].mean())

# compute cid and cid shuffle:
cid_, cid_shuff = CID(defects)

res_cid = {
    'zeta' : ar.zeta,
    'cid' : cid_,
    'cid_shuffle' : cid_shuff,
    'lambda' : 1. - cid_/cid_shuff
}

res_defect_density = {
    'zeta' : ar.zeta,
    'density' : defect_density
}

with open(f"/lustre/astro/robinboe/HD_CID/cid-results-2048/cid-results-L-2048-zeta-{zeta}-rep-{rep+1}.pkl", "wb") as file:
    pickle.dump(res_cid, file)

with open(f"/lustre/astro/robinboe/HD_CID/defect-density-results-2048/defect-density-results-L-2048-zeta-{zeta}-rep-{rep+1}.pkl", "wb") as file:
    pickle.dump(res_defect_density, file)

In [ ]:

def main(rep, zeta):
    main_cid(rep, zeta)

if __name__ == "__main__":
    zeta = float(sys.argv[1])
    rep = int(sys.argv[2])
    main(rep, zeta)
